<a href="https://colab.research.google.com/github/duartele/exerc-jupyternotebook/blob/main/Geospatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  geospatial file formats, such as shapefile, GeoJSON, KML, and GPKG. 

In [ ]:
import geopandas as gpd

In [ ]:
# Read in the data
full_data = gpd.read_file("../input/geospatial-learn-course-data/DEC_lands/DEC_lands/DEC_lands.shp")

# View the first five rows of the data
full_data.head()

In [ ]:
data = full_data.loc[:, ["CLASS", "COUNTY", "geometry"]].copy()